<img src="./files/allfed_logo.png">

# Agricultural Shipping Route Analysis: Feeding Ruminants with Residues

Developing effective approaches to feeding everyone in the event of a global catastrophe involves understanding how different agricultural products can be shipped within and between countries.
In this notebook I'll utilise code developed by ALLFED to build up a data representation of agricultural shipping routes, and use that model to perform analysis on the movement of livestock and agricultural residues.

## Table of Contents
1. [Goals](#Goals)
2. [Constraints and assumptions](#Constraints-and-assumptions)
3. [Preprocess route data](#1.-Preprocess-route-data)
3. [Preprocess livestock and crop data](#2.-Preprocess-livestock-and-crop-data)
3. [Herd livestock](#3.-Herd-livestock)
3. [Create connected network](#4.-Create-connected-network)
3. [Convert to graph and run analysis](#5.-Convert-to-graph-and-run-analysis)


## Goals

Our goal is to figure out the most efficient way to feed residues (what's left over after a crop has been harvested) to livestock, given the current spatial distribution of crops and livestock. Specifically, given our constraints and assumptions defined in the next section, we want to understand:

1. What quantity of livestock can be fed by residues, by either moving livestock to residues or vice versa
* The average distance livestock/residues will need to be transported
* What quantity of residue will need to be transported
* What quantity of livestock will need to be transported


The initial test data we will be using for this exercise is in Tasmania, Australia, and consists of rail data, road data, the location of cattle, and the location of cropland. We'll be using datasets which are available globally, meaning we can readily scale up the present analysis. 

Shipping route data | Livestock data | Cropland data
- | - | -
<img src="./files/tas_routes.png"> | <img src="./files/tas_livestock.png"> | <img src="./files/tas_crops.png">





The code used here was developed for the explicit purpose of doing global spatial analysis, and can be found at https://github.com/allfed/allfed-spatial 

## Constraints and assumptions

1. If a unit of cattle are within a certain distance of cropland (<= R), we'll move cattle to the cropland
* If a unit of cattle are outside a certain distance from cropland (> R), we'll move residues from the cropland to the livestock
* Each unit of cattle has a fixed consumption rate (C)
* Each unit of cropland has a fixed supply (S)
* Residues can be transported along existing railway and road routes an unlimited distance
* Cattle can be transported directly (as the crow flies) to cropland (as long as distance is < R)
* Cost per km. to transport residues along railway/road can be set in a rule-based manner

Do we also want to look at pasture?

##### Herd distance (R) 
[This page](https://en.wikipedia.org/wiki/Cattle_drives_in_the_United_States) suggests that cattle can be sustainably (weight maintained) herded overland at 24 km. / day. However, if we are herding with the intent of consuming all residues in an area, this value will be considerably smaller. Set to 1km as an initial value (to be updated).

##### Max herd distance (Rmax) 
There's likely an effective maximum distance we want to herd cattle in the first place. Set to 50km as an initial value (to be updated)

##### Consumption rate (C)
Set to 10kg per cow per day (to be updated). Allow adjustment of scenario time parameter L, which is number of days to feed cows.

##### Supply (S)
Set to 1000 kg per hectare (to update)

##### Herd time (L)
Maximum number of days to herd cattle, i.e. to allow cattle to feed within R/Rmax before transporting residues to them. Set to 365 days as an initial scenario.

## Setting up

In [18]:
# Allow access to Python imports from higher level folder
import sys
sys.path.append('..')

# Define directory locations
TEST_DIR = '../test/'
IN_SHAPE_DIR = '../test/in_shape/'
GRAPH_DIR = '../test/graph'
OUTPUT_DIR = '../test/output/'

# Define key variables
R = 1000 # maximum cattle herd distance in units metres per day (eating all day)
Rmax = 50000 # maximum distance to herd cattle
C = 10 # residue consumption by cattle in units of kg per cow per day
S = 1000 # supply of residue in units of kg per hectare (kg per 0.01 sq km)
L = 365 # number of days to feed cows

##  1. Preprocess route data

First we'll load in our route data and make sure it's all geometrically connected. It's important that routes are properly connected as we use these connections to infer traversible routes. 

Some source data describing (for example) roads will appear connected, but will actually be disconnected when zoomed in. To address this, we'll "snap" together line segments which are close to each other.

Road network | Zoomed in | Fixed via 'snapping'
- | - | -
<img src="./files/route_network_highlighted.png"> | <img src="./files/route_network_disconnect.png"> | <img src="./files/route_network_snapped.png">

In [19]:
import fiona
from shapely.geometry import LineString, shape
import copy

# ALLFED libs
from geometry.snap import snap_linestrings
from geometry.io import write_shape
from geometry.project import project_to_utm, project_from_utm
from geometry.line import split_line_by_distance, join_points_to_lines

In [20]:
# Load road shapefile (just look at roads for now)
ROAD_PATH = TEST_DIR + 'ne_10m_roads_test.shp'
roadF = fiona.open(ROAD_PATH)
road_geoms = [LineString(shape(f['geometry'])) for f in roadF]
road_data = [f['properties'] for f in roadF]

# Project to UTM coordinate system
road_geoms = project_to_utm(road_geoms, epsg='28355', zone=55)

# Create new snapped LineString collection
snapped = snap_linestrings(500, road_geoms) # snap line endpoints which are within 500 metres of each other

# Split out network every 10km to provide regularly spaced attachment points for our eventual network
split_lines = []
split_line_data = []
for i, s in enumerate(snapped):
    split = split_line_by_distance(s, 10000)
    for j in range(len(split)):
        split_line_data.append(copy.deepcopy(road_data[i]))
    split_lines.extend(split)

# Add length information
for i, rd in enumerate(split_line_data):
    rd['length_km'] = max(split_lines[i].length / 1000, 1)

# Write output to shapefile
test_output = project_from_utm(split_lines, epsg='28355', zone=55)
write_shape(test_output, split_line_data, roadF.schema, IN_SHAPE_DIR + 'routes.shp')

## 2. Preprocess livestock and crop data

In subsequent stages we'll first reference our livestock data to our crop data to determine which livestock can be moved directly to the location of residues. We'll then move all remaining residues to feed livestock which have not yet been fed.

In order to do this, we first need to convert our crop and livestock data to queryable points in space with attributes describing consumption and supply (required for the initial stage described above), and connect this data to our route network (required for the latter stage described above).

The livestock and crop data is in a "raster" format, consisting of pixels with assigned values representing livestock numbers, which we want to preserve. To query and connect this data to our network, we'll first convert each pixel into a spatial point at the pixel centroid, with data attributes indicating the consumption rate of livestock / supply of residue at that location

Initial data | Converted to points
- | -
<img src="./files/tas_livestock.png"> | <img src="./files/tas_livestock_points.png">

In [21]:
from collections import OrderedDict

# ALLFED libs
from raster.conversions import raster_to_points

In [22]:
# Load livestock and crop rasters

LIVESTOCK_PATH = TEST_DIR + 'cgiar_glb_cattle_cc2006_ad_test_resampled_01.tif'
CROP_PATH = TEST_DIR + 'earthstat_crop2000_5m_test.tif'

l_geoms, l_data = raster_to_points(LIVESTOCK_PATH)
l_geoms = project_to_utm(l_geoms, epsg='28355', zone=55)
c_geoms, c_data = raster_to_points(CROP_PATH)
c_geoms = project_to_utm(c_geoms, epsg='28355', zone=55)

# Add consuption rate to cattle
# Each pixel is 95 sq km
# Pixel values are head per sq. km.
for l in l_data:
    head = l['value'] * 95
    l['head'] = head
    l['demand'] = head * C * L # total demand in (kg / day) * 2 years
    l['remaining'] = l['demand']
    l['type'] = 'demand'

# Add supply to crops
# Each pixel is 65 sq km
# Pixel values are proportion of pixel dedicated to cropping
crop_factor = (S/0.01) * 65 # supply per square km. * square km. per pixel
for c in c_data:
    c['supply'] = c['value'] * crop_factor # total supply in kg.
    c['remaining'] = c['supply']
    c['type'] = 'supply'

## 3. Herd livestock

Now we'll "herd" our livestock, by having them gradually consume crop residues outwards over a series of time-steps, until we reach our maximum range.

In [23]:
import numpy as np
import rtree
import copy

# ALLFED libs
from operations.consume import consume

In [24]:
# From each livestock point, buffer out, consuming the crops encountered by each 
start = 100
end = min(R * L, Rmax) # (km / day) * total number of days, or max distance
step = (end - start)/10
buffer_steps = np.arange(start, end + step, step)

# Copy data from raster processing
crop_data = copy.deepcopy(c_data)
livestock_data = copy.deepcopy(l_data)

In [25]:
# Create and populate a spatial index for crops
crop_index = rtree.index.Index()
for c_id, c_geom in enumerate(c_geoms):
    crop_index.insert(c_id, c_geom.bounds)

for i, step in enumerate(buffer_steps):
    
    final_range_geoms = []
    
    # buffer livestock to this range
    bufferer = lambda p : p.buffer(step)
    livestock_ranges = map(bufferer, l_geoms)

    # iterate through 
    for l_id, l_range in enumerate(livestock_ranges):
        
        final_range_geoms.append(l_range)

        # get list of fids where bounding boxes intersect
        c_ids = [int(i) for i in crop_index.intersection(l_range.bounds)]

        # allocate supply
        for c_id in c_ids:
            c_out, _ = consume(crop_data[c_id], livestock_data[l_id])
            if c_out['remaining'] == 0:
                crop_index.delete(c_id, c_geoms[c_id].bounds)

In [26]:
# Put supply/demand info into a single diff field
for l in livestock_data:
    l['diff'] = l['remaining'] * -1

for c in crop_data:
    c['diff'] = c['remaining']

In [27]:
# Write shapefile output from this stage
DEMAND_POINT_PROPERTIES = {
    'value': 'float:16',
    'head': 'float:16',
    'demand': 'float:16',
    'remaining': 'float:16', 
    'type': 'str',
    'diff': 'float:16',
}

DEMAND_EDGE_PROPERTIES = {
    'value': 'float:16',
    'head': 'float:16',
    'demand': 'float:16',
    'remaining': 'float:16', 
    'type': 'str',
    'diff': 'float:16',
    'length_km': 'float:16'
}

SUPPLY_POINT_PROPERTIES = {
    'value': 'float:16', 
    'supply': 'float:16', 
    'remaining': 'float:16', 
    'type': 'str',
    'diff': 'float:16',
}

SUPPLY_EDGE_PROPERTIES = {
    'value': 'float:16', 
    'supply': 'float:16', 
    'remaining': 'float:16', 
    'type': 'str',
    'diff': 'float:16',
    'length_km': 'float:16'
}

schema = OrderedDict({
    'geometry': 'Point',
    'properties': DEMAND_POINT_PROPERTIES
})
test_output = project_from_utm(l_geoms, epsg='28355', zone=55)
write_shape(test_output, livestock_data, schema, OUTPUT_DIR + 'livestock_herd.shp')


schema = OrderedDict({
    'geometry': 'Point',
    'properties': SUPPLY_POINT_PROPERTIES
})
test_output = project_from_utm(c_geoms, epsg='28355', zone=55)
write_shape(test_output, crop_data, schema, OUTPUT_DIR + 'crops_herd.shp')


# schema = OrderedDict({
#     'geometry': 'Polygon',
#     'properties': {'value': 'float:16', 'head': 'float:16', 'demand': 'float:16', 'remaining': 'float:16'}
# })
# test_output = project_from_utm(final_range_geoms, epsg='28355', zone=55)
# write_shape(test_output, livestock_data, schema, TEST_DIR + 'livestock_range.shp')

## 4. Create connected network

We'll now connect the remaining sources of residues and unfed livestock to the network

In [28]:
# Remove any crop points which have been depleted of supply.
c_combined = list(filter(lambda c: c[0]['remaining'] > 0,list(zip(crop_data, c_geoms))))
crop_data_filtered, crop_geoms_filtered = map(list, zip(*c_combined))

# Remove any livestock points which have been depleted of demand.
l_combined = list(filter(lambda c: c[0]['remaining'] > 0,list(zip(livestock_data, l_geoms))))
livestock_data_filtered, livestock_geoms_filtered = map(list, zip(*l_combined))

# Join our livestock points to the network
l_joins = join_points_to_lines(livestock_geoms_filtered, split_lines)
c_joins = join_points_to_lines(crop_geoms_filtered, split_lines)

# Add length information
for i, ld in enumerate(livestock_data_filtered):
    ld['length_km'] = max(l_joins[i].length / 1000, 1)

for i, cd in enumerate(crop_data_filtered):
    cd['length_km'] = max(c_joins[i].length / 1000, 1)

# Write shapefile output from this stage
schema = OrderedDict({
    'geometry': 'Point',
    'properties': DEMAND_EDGE_PROPERTIES
})
test_output = project_from_utm(livestock_geoms_filtered, epsg='28355', zone=55)
write_shape(test_output, livestock_data_filtered, schema, IN_SHAPE_DIR + 'livestock_points.shp')


schema = OrderedDict({
    'geometry': 'Point',
    'properties': SUPPLY_EDGE_PROPERTIES
})
test_output = project_from_utm(crop_geoms_filtered, epsg='28355', zone=55)
write_shape(test_output, crop_data_filtered, schema, IN_SHAPE_DIR + 'crop_points.shp')


schema = OrderedDict({
    'geometry': 'LineString',
    'properties': DEMAND_EDGE_PROPERTIES
})
test_output = project_from_utm(l_joins, epsg='28355', zone=55)
write_shape(test_output, livestock_data_filtered, schema, IN_SHAPE_DIR + 'livestock_edges.shp')


schema = OrderedDict({
    'geometry': 'LineString',
    'properties': SUPPLY_EDGE_PROPERTIES
})
test_output = project_from_utm(c_joins, epsg='28355', zone=55)
write_shape(test_output, crop_data_filtered, schema, IN_SHAPE_DIR + 'crop_edges.shp')

## 5. Convert to graph and run analysis

We'll now convert our processed data into a [graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)) - a mathematical structure consisting of nodes and edges. This structure allows us to easily perform analysis on our data.

In [29]:
import networkx as nx

In [30]:
# Read back in as a graph, then write again to inspect
G = nx.read_shp(IN_SHAPE_DIR).to_undirected()
nx.write_shp(G, GRAPH_DIR)

In [31]:
nx.is_connected(G)

True